# Query With Filtered IDs

Chroma now supports passing a list of IDs at query time to shrink the search space, and allowing for better interoperability with other databases.

This notebook will go over an example using sqlite as a primary database, joining tables to find matching IDs to then compute semantic search across.

In [1]:
! pip install chromadb --quiet
! pip install pandas --quiet
import chromadb
import sqlite3
from datetime import datetime, timedelta
from chromadb.api import ClientAPI
from chromadb.utils.embedding_functions import DefaultEmbeddingFunction
import random
import pandas as pd



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


## Setup Databases and Sample Data

This is a generated dataset of 100 products and descriptions that fall into one of 5 categories: Electronics, Clothing, Home, Books, Sports. It creates 2 tables in sqlite, one for the categories and the other for the product, along with necessary metadata and the foreign key referencing the category.

While Chroma is capable of metadata storage and filtering which would also solve this case, this is a hypothetical scenario, where you can imagine a user did not want to double store their data, and rather only wanted to store unstructured text data with Chroma.

### Sample Data

In [2]:
jurisdictions = {
    1: "California Supreme Court",
    2: "New York Federal District Court",
    3: "Texas State Court",
    4: "Florida Appellate Court",
    5: "Illinois Circuit Court",
    6: "Massachusetts Superior Court",
    7: "Washington State Court",
    8: "Georgia Court of Appeals",
    9: "Pennsylvania Commonwealth Court",
    10: "Ohio District Court"
}

case_types = {
    1: "Civil",
    2: "Criminal",
    3: "Constitutional",
    4: "Administrative",
    5: "Family"
}

case_names = {
    1: [  # Civil cases
        "Smith v. Johnson", "Brown v. Corporation", "Davis v. Insurance Co",
        "Wilson v. Hospital", "Taylor v. Construction", "Anderson v. Retail",
        "Thomas v. Manufacturing", "Jackson v. Technology", "White v. Healthcare",
        "Harris v. Education", "Martin v. Transportation", "Thompson v. Energy",
        "Garcia v. Real Estate", "Martinez v. Entertainment", "Robinson v. Finance",
        "Clark v. Agriculture", "Rodriguez v. Telecommunications", "Lewis v. Defense",
        "Lee v. Pharmaceuticals", "Walker v. Environmental"
    ],
    2: [  # Criminal cases
        "State v. Williams", "People v. Miller", "Commonwealth v. Davis",
        "State v. Garcia", "People v. Rodriguez", "Commonwealth v. Martinez",
        "State v. Robinson", "People v. Clark", "Commonwealth v. Lewis",
        "State v. Lee", "People v. Walker", "Commonwealth v. Hall",
        "State v. Allen", "People v. Young", "Commonwealth v. Hernandez",
        "State v. King", "People v. Wright", "Commonwealth v. Lopez",
        "State v. Hill", "People v. Scott"
    ],
    3: [  # Constitutional cases
        "Citizens v. Government", "Association v. State", "Union v. Legislature",
        "Coalition v. Executive", "Committee v. Administration", "Group v. Agency",
        "Organization v. Department", "Alliance v. Commission", "Council v. Board",
        "Federation v. Authority", "League v. Bureau", "Society v. Division",
        "Institute v. Office", "Foundation v. Service", "Center v. Program",
        "Network v. Unit", "Partnership v. Section", "Corporation v. Branch",
        "Enterprise v. Division", "Institution v. Department"
    ],
    4: [  # Administrative cases
        "Petitioner v. Agency", "Applicant v. Commission", "Claimant v. Board",
        "Appellant v. Department", "Respondent v. Authority", "Complainant v. Bureau",
        "Plaintiff v. Division", "Defendant v. Office", "Party v. Service",
        "Individual v. Program", "Organization v. Unit", "Company v. Section",
        "Corporation v. Branch", "Association v. Division", "Group v. Department",
        "Committee v. Agency", "Council v. Commission", "Board v. Authority",
        "Commission v. Bureau", "Authority v. Division"
    ],
    5: [  # Family cases
        "Family v. Support", "Parent v. Custody", "Guardian v. Adoption",
        "Spouse v. Divorce", "Child v. Protection", "Relative v. Guardianship",
        "Partner v. Visitation", "Mother v. Paternity", "Father v. Maintenance",
        "Sibling v. Inheritance", "Heir v. Estate", "Beneficiary v. Trust",
        "Dependent v. Support", "Ward v. Care", "Minor v. Protection",
        "Elder v. Care", "Disabled v. Support", "Vulnerable v. Protection",
        "Dependent v. Assistance", "Family v. Welfare"
    ]
}

case_descriptions = {
    1: [  # Civil cases
        "Landmark case establishing precedent for corporate liability in environmental damage claims.",
        "Pivotal decision on insurance coverage for natural disaster claims and policy interpretation.",
        "Significant ruling on medical malpractice and hospital liability standards.",
        "Important case on construction defect claims and contractor responsibility.",
        "Notable decision on retail consumer protection and product liability.",
        "Key ruling on manufacturing safety standards and workplace regulations.",
        "Major case on technology patent infringement and intellectual property rights.",
        "Critical decision on healthcare provider obligations and patient rights.",
        "Essential ruling on educational institution responsibilities and student safety.",
        "Significant case on transportation safety regulations and operator liability.",
        "Important decision on energy company regulations and environmental compliance.",
        "Notable ruling on real estate transaction disputes and disclosure requirements.",
        "Key case on entertainment industry contracts and performer rights.",
        "Major decision on financial institution regulations and consumer protection.",
        "Critical ruling on agricultural practices and environmental impact.",
        "Essential case on telecommunications regulations and consumer rights.",
        "Significant decision on defense contract disputes and government liability.",
        "Important ruling on pharmaceutical product liability and safety standards.",
        "Notable case on environmental regulations and corporate compliance.",
        "Key decision on healthcare provider standards and patient care requirements."
    ],
    2: [  # Criminal cases
        "Landmark case establishing new standards for criminal procedure and evidence handling.",
        "Pivotal decision on criminal sentencing guidelines and judicial discretion.",
        "Significant ruling on criminal defense rights and legal representation.",
        "Important case on criminal investigation procedures and police conduct.",
        "Notable decision on criminal trial procedures and jury selection.",
        "Key ruling on criminal evidence admissibility and chain of custody.",
        "Major case on criminal sentencing and rehabilitation programs.",
        "Critical decision on criminal appeal procedures and appellate rights.",
        "Essential ruling on criminal defense strategies and legal representation.",
        "Significant case on criminal investigation techniques and evidence collection.",
        "Important decision on criminal trial procedures and witness testimony.",
        "Notable ruling on criminal sentencing and parole eligibility.",
        "Key case on criminal defense rights and legal representation.",
        "Major decision on criminal investigation procedures and evidence handling.",
        "Critical ruling on criminal trial procedures and jury instructions.",
        "Essential case on criminal sentencing guidelines and judicial discretion.",
        "Significant decision on criminal appeal procedures and appellate rights.",
        "Important ruling on criminal defense strategies and legal representation.",
        "Notable case on criminal investigation techniques and evidence collection.",
        "Key decision on criminal trial procedures and witness testimony."
    ],
    3: [  # Constitutional cases
        "Landmark case establishing precedent for constitutional rights and government powers.",
        "Pivotal decision on constitutional interpretation and judicial review.",
        "Significant ruling on constitutional rights and individual liberties.",
        "Important case on constitutional powers and government authority.",
        "Notable decision on constitutional interpretation and legal precedent.",
        "Key ruling on constitutional rights and civil liberties.",
        "Major case on constitutional powers and government limitations.",
        "Critical decision on constitutional interpretation and judicial authority.",
        "Essential ruling on constitutional rights and individual freedoms.",
        "Significant case on constitutional powers and government responsibilities.",
        "Important decision on constitutional interpretation and legal standards.",
        "Notable ruling on constitutional rights and civil protections.",
        "Key case on constitutional powers and government obligations.",
        "Major decision on constitutional interpretation and judicial discretion.",
        "Critical ruling on constitutional rights and individual protections.",
        "Essential case on constitutional powers and government duties.",
        "Significant decision on constitutional interpretation and legal principles.",
        "Important ruling on constitutional rights and civil rights.",
        "Notable case on constitutional powers and government functions.",
        "Key decision on constitutional interpretation and judicial authority."
    ],
    4: [  # Administrative cases
        "Landmark case establishing precedent for administrative law and agency authority.",
        "Pivotal decision on administrative procedures and regulatory compliance.",
        "Significant ruling on administrative powers and government agencies.",
        "Important case on administrative law and regulatory enforcement.",
        "Notable decision on administrative procedures and agency discretion.",
        "Key ruling on administrative powers and government authority.",
        "Major case on administrative law and regulatory standards.",
        "Critical decision on administrative procedures and agency powers.",
        "Essential ruling on administrative powers and government functions.",
        "Significant case on administrative law and regulatory requirements.",
        "Important decision on administrative procedures and agency authority.",
        "Notable ruling on administrative powers and government responsibilities.",
        "Key case on administrative law and regulatory compliance.",
        "Major decision on administrative procedures and agency discretion.",
        "Critical ruling on administrative powers and government authority.",
        "Essential case on administrative law and regulatory standards.",
        "Significant decision on administrative procedures and agency powers.",
        "Important ruling on administrative powers and government functions.",
        "Notable case on administrative law and regulatory requirements.",
        "Key decision on administrative procedures and agency authority."
    ],
    5: [  # Family cases
        "Landmark case establishing precedent for child custody and parental rights.",
        "Pivotal decision on child support guidelines and parental obligations.",
        "Significant ruling on adoption procedures and parental rights.",
        "Important case on divorce proceedings and marital property division.",
        "Notable decision on child protection and welfare services.",
        "Key ruling on guardianship arrangements and minor protection.",
        "Major case on visitation rights and parental access.",
        "Critical decision on paternity establishment and parental rights.",
        "Essential ruling on child maintenance and support obligations.",
        "Significant case on inheritance rights and estate distribution.",
        "Important decision on trust administration and beneficiary rights.",
        "Notable ruling on dependent support and family assistance.",
        "Key case on child care arrangements and parental responsibilities.",
        "Major decision on minor protection and welfare services.",
        "Critical ruling on elder care and family support.",
        "Essential case on disability support and family assistance.",
        "Significant decision on vulnerable person protection and family care.",
        "Important ruling on dependent assistance and family support.",
        "Notable case on family welfare and support services.",
        "Key decision on child protection and family assistance."
    ]
}

### Instantiate Databases and Insert Data

In [3]:
def setup_databases(conn, client: ClientAPI):
    """Set up SQLite and ChromaDB with legal case data."""
    print("Setting up databases...")
    
    # Create SQLite database
    cursor = conn.cursor()
    
    # Create tables
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS jurisdictions (
        id INTEGER PRIMARY KEY,
        name TEXT
    )
    ''')
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS case_types (
        id INTEGER PRIMARY KEY,
        name TEXT
    )
    ''')
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS cases (
        id INTEGER PRIMARY KEY,
        name TEXT,
        jurisdiction_id INTEGER,
        case_type_id INTEGER,
        filing_date TEXT,
        description TEXT,
        FOREIGN KEY (jurisdiction_id) REFERENCES jurisdictions(id),
        FOREIGN KEY (case_type_id) REFERENCES case_types(id)
    )
    ''')
    
    # Insert jurisdictions
    for i, jurisdiction in jurisdictions.items():
        cursor.execute("INSERT INTO jurisdictions VALUES (?, ?)", (i, jurisdiction))
    
    # Insert case types
    for i, case_type in case_types.items():
        cursor.execute("INSERT INTO case_types VALUES (?, ?)", (i, case_type))
    
    # Generate random dates within the last 10 years
    end_date = datetime.now()
    start_date = end_date - timedelta(days=3650)  # 10 years
    
    # Insert cases
    case_id = 1
    case_texts = []
    case_ids = []
    
    ef = DefaultEmbeddingFunction()
    
    for case_type_id in range(1, 6):
        for i, case_name in enumerate(case_names[case_type_id]):
            description = case_descriptions[case_type_id][i] if i < len(case_descriptions[case_type_id]) else "Case description not available."
            
            # Generate random jurisdiction
            jurisdiction_id = random.randint(1, 10)
            
            # Generate random filing date
            random_days = random.randint(0, 3650)
            filing_date = (start_date + timedelta(days=random_days)).strftime('%Y-%m-%d')
            
            # Insert case into database
            cursor.execute(
                "INSERT INTO cases VALUES (?, ?, ?, ?, ?, ?)",
                (case_id, case_name, jurisdiction_id, case_type_id, filing_date, description)
            )
            
            case_ids.append(str(case_id))
            case_id += 1
            case_texts.append(f"{case_name}: {description}")
    
    # Commit changes
    conn.commit()
    
    # Create ChromaDB collection
    collection = client.get_or_create_collection(
        name="legal_case_embeddings",
    )
    
    
    collection.add(
        ids=case_ids,
        embeddings=ef(case_texts),
    )
    
    print(f"Database setup complete. Added {case_id-1} cases across {len(jurisdictions)} jurisdictions and {len(case_types)} case types.")


In [4]:
conn = sqlite3.connect(':memory:')
client = chromadb.EphemeralClient()

setup_databases(conn, client)

Setting up databases...
Database setup complete. Added 100 cases across 10 jurisdictions and 5 case types.


In [5]:
def view_sqlite_data(conn):
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM jurisdictions")
    df = pd.DataFrame(cursor.fetchall(), columns=['id', 'name'])
    print("\nJurisdictions:")
    print(df)
    
    cursor.execute("SELECT * FROM case_types")
    df = pd.DataFrame(cursor.fetchall(), columns=['id', 'name'])
    print("\nCase Types:")
    print(df)
    
    cursor.execute("""
        SELECT c.id, c.name, j.name as jurisdiction, ct.name as case_type, 
               c.filing_date, c.description
        FROM cases c
        JOIN jurisdictions j ON c.jurisdiction_id = j.id
        JOIN case_types ct ON c.case_type_id = ct.id
    """)
    df = pd.DataFrame(cursor.fetchall(), 
                     columns=['id', 'name', 'jurisdiction', 'case_type', 
                             'filing_date', 'description'])
    print("\nCases:")
    print(df)

## Querying using Filtered IDs
We will now use the sqlite to first fetch the list of ids we would want to query on in 2 scenarios.

Scenario 1: The user asks to see eco-friendly products in the Electronics category. We will fetch IDs corresponding to the Electronics category via foreign key, and use the set of IDs to shrink our search space.

Scenario 2: The user wants to find items of a specific price across multiple categories. This will utilize more complex filtering in sqlite to then narrow our search space before doing semantic search.

In [8]:
def demo_filtered_search(conn, chroma_client: ClientAPI):
    cursor = conn.cursor()
    collection = chroma_client.get_collection(
        name="legal_case_embeddings",
    )
    
    # SCENARIO: Find relevant precedent cases in a specific jurisdiction within the last 10 years
    print("\nSCENARIO: Find relevant precedent cases in New York Federal District Court within the last 10 years\
        about corporate liability damage claims\n")
    
    # Step 1: Get cases matching structured criteria using SQL
    cursor.execute("""
        SELECT c.id 
        FROM cases c
        JOIN jurisdictions j ON c.jurisdiction_id = j.id
        WHERE j.name = 'New York Federal District Court'
        AND c.filing_date >= date('now', '-10 years')
    """)
    
    filtered_case_ids = [str(row[0]) for row in cursor.fetchall()]
    
    print(f"Found {len(filtered_case_ids)} cases in New York Federal District Court from the last 10 years")
    
    # Step 2: Use these IDs to filter ChromaDB search and organize by semantic relevance
    query_text = "corporate liability damage claims"
    
    results = collection.query(
        query_texts=[query_text],
        ids=filtered_case_ids,
        n_results=3,
    )
    
    # Step 3: Get full case details from SQLite
    print("\nResults (relevant precedent cases in New York Federal District Court):")
    
    for i, case_id in enumerate(results['ids'][0]):
        cursor.execute("""
            SELECT c.name, j.name, ct.name, c.filing_date, c.description
            FROM cases c
            JOIN jurisdictions j ON c.jurisdiction_id = j.id
            JOIN case_types ct ON c.case_type_id = ct.id
            WHERE c.id = ?
        """, (case_id,))
        
        case_name, jurisdiction, case_type, filing_date, description = cursor.fetchone()
        print(f"\n{i+1}. {case_name} - {jurisdiction} ({filing_date})")
        print(f"   Type: {case_type}")
        print(f"   Description: {description}")


In [9]:
demo_filtered_search(conn, client)


SCENARIO: Find relevant precedent cases in New York Federal District Court within the last 10 years        about corporate liability damage claims

Found 17 cases in New York Federal District Court from the last 10 years

Results (relevant precedent cases in New York Federal District Court):

1. Wilson v. Hospital - New York Federal District Court (2020-08-05)
   Type: Civil
   Description: Important case on construction defect claims and contractor responsibility.

2. Anderson v. Retail - New York Federal District Court (2016-01-04)
   Type: Civil
   Description: Key ruling on manufacturing safety standards and workplace regulations.

3. Martin v. Transportation - New York Federal District Court (2022-09-26)
   Type: Civil
   Description: Important decision on energy company regulations and environmental compliance.
